In [1]:
#!sudo pip install flask
#import sys
#!{sys.executable} -m pip install flask

In [23]:
import json
import joblib
from flask import Flask, request
import numpy as np
from tensorflow.keras.models import load_model

# Load labels
from io import BytesIO
from tensorflow.python.lib.io import file_io

In [26]:
# Load model into object
model = load_model('model-e5.h5')

In [34]:
# Load result into array
file = open('X-200', 'rb')
X = np.load(file, allow_pickle=True)
file.close

y = np.load('Y-200')

In [2]:
app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, world!"

# Create /predict endpoint with POST method
@app.route('/predict', methods=['POST'])
def predict():
    request_json = request.json
    print("data: {}").format(request_json)
    print("type: {}").format(type(request_json))
    
    # Predict from 'data' field in request body,
    # and return prediction in response body
    prediction = model.predict(request_json.get('data'))
    prediction_string = [str(d) for d in prediction]
    response_json = {
        'data' : request_json.get('data'),
        'prediction' : list(prediction_string)
    }
    
    return json.dumps(response_json)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
    #app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.148.0.2:5000/ (Press CTRL+C to quit)


##### Prediction label based using model.fit

In [ ]:
# 80:20 for split size
size = int(len(X) * 0.8)

x_train, x_test = X[:size], X[size:]
y_train, y_test = y[:size], y[size:]

In [ ]:
# Free unused memories
import gc
del X
del y
del x_train
del y_train
gc.collect()

In [ ]:
# Evaluate the model on test data
results = model_faces.evaluate(x_test, y_test, batch_size=128)

# Generate predictions
# For example, generate predictions for 3 samples
predictions = model_faces.predict(x_test[:3])
print(predictions.shape)

##### Prediction probabilities using face processing (X)

In [ ]:
# Prediction probability of face using X
filenames = '../resources/dataset/test_videos/*.mp4'

# Prediction
bias = -0.4
weight = 0.68235746

probabilities = []
for filename, x_i in zip(filenames, X):
    if x_i is not None:
        prob = 1 / (1 + np.exp(-(bias + (weight * x_i).mean())))
    else:
        prob = 0.5
    probabilities.append([os.path.basename(filename), prob])